## 모듈 호출

In [38]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, classification_report, f1_score

import lightgbm as lgb


C:\Users\korea\Anaconda3\envs\DL\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## 데이터 불러오기

In [3]:
train = pd.read_csv("../3.통합/7.시작 주 기준 펼치기/train_start_spread.csv")
test = pd.read_csv("../3.통합/7.시작 주 기준 펼치기/test_start_spread.csv")
train_label = pd.read_csv("train_label.csv")

In [4]:
train.shape

(100000, 586)

## 데이터 merge(라벨과 합치기)

In [5]:
train = pd.merge(train, train_label, on='acc_id',how='inner')

In [18]:
## labeling
def change(x):
    if x == 'week':
        return 0
    elif x == 'month':
        return 1
    elif x == '2month':
        return 2
    else:
        return 3

train['label'] = train.label.apply(change)

## 피쳐, 라벨 구분

In [44]:
X = train.drop(['acc_id', 'label'], axis=1).values
y = train.label.values

## 데이터 분할

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=123, 
                                                    stratify=y)

# Model

# 모든 classifier model을 고려하여 가장 좋은 모델을 selection 해보자

# Model1

# 딥러닝 : 3 Hidden Layer MLP & Adam Optimizer & Dropout & Early stopping

In [32]:
X_train.shape

(70000, 585)

In [22]:
depth = len(np.unique(y))

y_train = tf.one_hot(y_train, depth=depth).eval(session = tf.Session())
y_test = tf.one_hot(y_test, depth=depth).eval(session = tf.Session())

In [35]:
tf.reset_default_graph()

keep_prob = tf.placeholder(tf.float32)
#input layer
X = tf.placeholder(tf.float32, shape=[None, 585])
Y = tf.placeholder(tf.float32, shape=[None, 4])
#first hidden layer
W1_ = tf.get_variable("W1_", [585, 50], initializer = tf.contrib.layers.xavier_initializer())
b1_ = tf.get_variable("b1_", [50], initializer = tf.zeros_initializer())
L1 = tf.nn.relu(tf.matmul(X, W1_) + b1_)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
#second hidden layer
W2_ = tf.get_variable("W2_", [50, 30], initializer = tf.contrib.layers.xavier_initializer())
b2_ = tf.get_variable("b2_", [30], initializer = tf.zeros_initializer())
L2 = tf.nn.relu(tf.matmul(L1, W2_) + b2_)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)
#third hidden layer
W3_ = tf.get_variable("W3_", [30, 10], initializer = tf.contrib.layers.xavier_initializer())
b3_ = tf.get_variable("b3_", [10], initializer = tf.zeros_initializer())
L3 = tf.nn.relu(tf.matmul(L2, W3_) + b3_)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
#output layer
W4_ = tf.get_variable("W4_", [10, 4], initializer = tf.contrib.layers.xavier_initializer())
b4_ = tf.get_variable("b4_", [4], initializer = tf.zeros_initializer())
hypothesis = tf.matmul(L3, W4_) + b4_

In [36]:
#learning rate
learning_rate = 0.01

In [37]:
#cost function & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [38]:
#Normalizer
from sklearn.preprocessing import Normalizer
scale = Normalizer()
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)

In [42]:
# initialize
init = tf.global_variables_initializer()

# train hyperparameters
epoch_cnt = 500
batch_size = 10000
iteration = len(X_train) // batch_size
saver = tf.train.Saver()
earlystop_threshold = 5
earlystop_cnt = 0

# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    prev_train_acc = 0.0
    max_val_acc = 0.0
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            _, loss = sess.run([optimizer, cost], 
                               feed_dict={X: X_train[start: end], Y: y_train[start: end], keep_prob : 0.7})
            start += batch_size; end += batch_size
            # Compute average loss
            avg_loss += loss / iteration
        
        if epoch % 100 == 0:
            
            # Validate model
            preds = tf.nn.softmax(hypothesis)  # Apply softmax to logits
            correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
            # Calculate accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            cur_train_acc = accuracy.eval({X: X_train, Y: y_train ,keep_prob: 1.0})
            cur_val_acc = accuracy.eval({X: X_test, Y: y_test ,keep_prob: 1.0})
            cur_val_loss = cost.eval({X: X_test, Y: y_test ,keep_prob: 1.0})

            print("epoch: "+str(epoch + 100)+
              ", train acc: " + str(cur_train_acc) +
              ", val acc: " + str(cur_val_acc) )
              #', train loss: '+str(avg_loss)+
              #', val loss: '+str(cur_val_loss))
            if cur_val_acc < max_val_acc:
                if cur_train_acc > prev_train_acc or cur_train_acc > 0.99:
                    if earlystop_cnt == earlystop_threshold:
                        print("early stopped on "+str(epoch))
                        break
                    else:
                        print("overfitting warning: "+str(earlystop_cnt))
                    earlystop_cnt += 1
                else:
                    earlystop_cnt = 0
            else:
                earlystop_cnt = 0
                max_val_acc = cur_val_acc
            # Save the variables to file.
            save_path = saver.save(sess, "model/model.ckpt")
            prev_train_acc = cur_train_acc

epoch: 100, train acc: 0.4709857, val acc: 0.47166666
epoch: 200, train acc: 0.65775716, val acc: 0.65003335
epoch: 300, train acc: 0.6663857, val acc: 0.65566665
epoch: 400, train acc: 0.6677143, val acc: 0.6583667
epoch: 500, train acc: 0.6713714, val acc: 0.65966666


In [43]:
# Start testing
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "model/model.ckpt")
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("[Test Accuracy] :", accuracy.eval({X: X_test, Y: y_test, keep_prob: 1.0}))

INFO:tensorflow:Restoring parameters from model/model.ckpt
[Test Accuracy] : 0.65966666


In [8]:
import numpy as np
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

# Model2 : SVM - 가우지안 커널

In [ ]:
model1 = SVC(kernel = 'rbf')
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)
print(f1_score(y_pred, y_test, average='weighted'))

# Model3 : Random Forest

In [13]:
model2 = RandomForestClassifier(n_jobs=-1, random_state = 201) #사용할 모델
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)
print(f1_score(y_pred, y_test, average='weighted'))

0.715043730412898


In [15]:
from sklearn.linear_model import LogisticRegression

# Model4 : LogisticRegression

In [16]:
model3 = LogisticRegression(solver='lbfgs', n_jobs=-1)
model3.fit(X_train, y_train)
y_pred = model3.predict(X_test)
print(f1_score(y_pred, y_test, average='weighted'))

0.47617197756021656


# Model5 : XGBoost

In [17]:
model4 = XGBClassifier(random_state = 201, n_jobs=-1)
model4.fit(X_train, y_train)
y_pred = model4.predict(X_test)
print(f1_score(y_pred, y_test, average='weighted'))

0.6884774133054054


C:\Users\korea\Anaconda3\envs\DL\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Model6 : KNN

In [18]:
from sklearn.neighbors import KNeighborsClassifier

In [19]:
model5 = KNeighborsClassifier(n_jobs=-1) #default
model5.fit(X_train, y_train)
y_pred = model5.predict(X_test)
print(f1_score(y_pred, y_test, average='weighted'))

0.6758639327415544


In [20]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Model7 : LDA

In [21]:
model6 = LinearDiscriminantAnalysis()
model6.fit(X_train, y_train)
y_pred = model6.predict(X_test)
print(f1_score(y_pred, y_test, average='weighted'))

C:\Users\korea\Anaconda3\envs\DL\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.6056211552482658


# Model8 : QDA

In [22]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [23]:
model7 = QuadraticDiscriminantAnalysis()
model7.fit(X_train, y_train)
y_pred = model7.predict(X_test)
print(f1_score(y_pred, y_test, average='weighted'))

C:\Users\korea\Anaconda3\envs\DL\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\korea\Anaconda3\envs\DL\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\korea\Anaconda3\envs\DL\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\korea\Anaconda3\envs\DL\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.47673299167763794


# Model9 : GaussianNB

In [24]:
from sklearn.naive_bayes import GaussianNB

In [25]:
model8 = GaussianNB()
model8.fit(X_train, y_train)
y_pred = model8.predict(X_test)
print(f1_score(y_pred, y_test, average='weighted'))

0.4273648571416887


# Model10 : BaggingClassifier

In [9]:
model9 = BaggingClassifier(n_jobs=-1)
model9.fit(X_train, y_train)
y_pred = model9.predict(X_test)
print(f1_score(y_pred, y_test, average='weighted'))

0.710735657695165


# Final Model is Random Forest. So Let's tune hyperparameter of this model

# Hyper parameter Tuning

### 규칙
#### 1. 씨드값을 고정한다.
#### 2. 성능에 가장 큰 영향을 미치는 n_estimator(의사결정나무의 갯수)의 가장 좋은 값을 찾는다.
#### 3. 가장 좋은  n_estimator에 대하여 과대적합 방지를 위한 tree의 깊이인 max_depth가 보다 깊어 지지 않게 하여 오버피팅을 방지한다.
#### 4. 이렇게 결정한  n_estimator,  max_depth를 가지고 트리의 피쳐를 얼마나 무작위 될지 결정하는 max_features에 대한 best tune를 찾는다. 
#### 5. 마지막으로 가지를 분할하는 데 필요한 최소 샘플 수인 min_samples_split, 각 가지에서 필요한 최소 샘플 수인 min_samples_leaf를 결정하여 최종 파라미터 결정

In [15]:
model2 = RandomForestClassifier(max_depth=500, n_estimators=1000 , n_jobs=-1, random_state = 201) #사용할 모델
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_pred, y_test))

0.7304333333333334


In [16]:
rf = RandomForestClassifier(max_depth=500, n_estimators=1500 , n_jobs=-1, random_state = 201) #사용할 모델
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_pred, y_test))

0.7303333333333333


In [17]:
rf = RandomForestClassifier(max_depth=500, n_estimators=1700 , n_jobs=-1, random_state = 201) #사용할 모델
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_pred, y_test))

0.7305666666666667


# 2. 성능에 가장 큰 영향을 미치는 n_estimator(의사결정나무의 갯수)의 가장 좋은 값을 찾는다.

In [23]:
for i in np.linspace(1500, 2000, 10):
    n_estimator = int(i)
    rf = RandomForestClassifier(max_depth=500, n_estimators=n_estimator , n_jobs=-1, random_state = 201) #사용할 모델
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    f1 = f1_score(y_pred, y_test, average='weighted')
    
    print('n_estimator : {0} & F1 score : {1}' .format(n_estimator, f1))

n_estimator : 1500 & F1 score : 0.7343033712215249
n_estimator : 1555 & F1 score : 0.7341987415448632
n_estimator : 1611 & F1 score : 0.734342960539952
n_estimator : 1666 & F1 score : 0.7345018859720268
n_estimator : 1722 & F1 score : 0.7345090439599259
n_estimator : 1777 & F1 score : 0.7346516370076865
n_estimator : 1833 & F1 score : 0.7347312583359263
n_estimator : 1888 & F1 score : 0.7346781520200154
n_estimator : 1944 & F1 score : 0.7348209427485223
n_estimator : 2000 & F1 score : 0.7347995950679171


# 3. 가장 좋은 n_estimator에 대하여 과대적합 방지를 위한 tree의 깊이인 max_depth가 보다 깊어 지지 않게 하여 오버피팅을 방지한다.

In [24]:
for i in np.linspace(500, 1000, 10):
    depth = int(i)
    rf = RandomForestClassifier(max_depth=depth, n_estimators=1950, n_jobs=-1, random_state = 201) #사용할 모델
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    f1 = f1_score(y_pred, y_test, average='weighted')
    
    print('max_depth : {0} & F1 score : {1}' .format(depth, f1))

max_depth : 500 & F1 score : 0.7349295434328805
max_depth : 555 & F1 score : 0.7349295434328805
max_depth : 611 & F1 score : 0.7349295434328805


KeyboardInterrupt: 

# 4. 이렇게 결정한 n_estimator, max_depth를 가지고 트리의 피쳐를 얼마나 무작위 될지 결정하는 max_features에 대한 best tune를 찾는다.

In [27]:
int(np.sqrt(X_train.shape[1]))

24

In [30]:
for i in np.linspace(10, int(np.sqrt(X_train.shape[1])), 10):
    max_feature = int(i)
    rf = RandomForestClassifier(max_depth=500, n_estimators=1950, max_features = max_feature,n_jobs=-1, random_state = 201) #사용할 모델
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    f1 = f1_score(y_pred, y_test, average='weighted')
    
    print('max_features : {0} & F1 score : {1}' .format(max_feature, f1)) 

max_features : 10 & F1 score : 0.7335874743263953
max_features : 11 & F1 score : 0.7335536231780987
max_features : 13 & F1 score : 0.7335228154279132
max_features : 14 & F1 score : 0.7342896150333382
max_features : 16 & F1 score : 0.7340788560541349
max_features : 17 & F1 score : 0.7341077791584522
max_features : 19 & F1 score : 0.7340073122981773
max_features : 20 & F1 score : 0.7350169859481749
max_features : 22 & F1 score : 0.734694900575669
max_features : 24 & F1 score : 0.7349295434328805


In [31]:
for i in np.linspace(int(np.sqrt(X_train.shape[1])), X_train.shape[1], 100):
    max_feature = int(i)
    rf = RandomForestClassifier(max_depth=500, n_estimators=1950, max_features = max_feature,n_jobs=-1, random_state = 201) #사용할 모델
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    f1 = f1_score(y_pred, y_test, average='weighted')
    
    print('max_features : {0} & F1 score : {1}' .format(max_feature, f1))

max_features : 24 & F1 score : 0.7349295434328805
max_features : 29 & F1 score : 0.7349957954850568
max_features : 35 & F1 score : 0.7348533816871904
max_features : 41 & F1 score : 0.7345235398975106
max_features : 46 & F1 score : 0.7356165631994748
max_features : 52 & F1 score : 0.7357253096300669
max_features : 58 & F1 score : 0.7356065428985181
max_features : 63 & F1 score : 0.7358525987530192
max_features : 69 & F1 score : 0.7361901560522929
max_features : 75 & F1 score : 0.7362587985536095
max_features : 80 & F1 score : 0.7358568939804893
max_features : 86 & F1 score : 0.735798295436545
max_features : 92 & F1 score : 0.7357331045459541
max_features : 97 & F1 score : 0.735392435401977
max_features : 103 & F1 score : 0.735949544345767
max_features : 109 & F1 score : 0.735909092792204
max_features : 114 & F1 score : 0.735140889535815
max_features : 120 & F1 score : 0.7355326745626618
max_features : 126 & F1 score : 0.7358878931884424
max_features : 131 & F1 score : 0.7354983611540978

KeyboardInterrupt: 

In [32]:
rf = RandomForestClassifier(max_depth=100, n_estimators=1950, max_features =75, n_jobs=-1, random_state = 201) #사용할 모델
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
f1 = f1_score(y_pred, y_test, average='weighted')

In [35]:
f1 #best score

0.7362587985536095

# 5. 마지막으로 가지를 분할하는 데 필요한 최소 샘플 수인 min_samples_split, 각 가지에서 필요한 최소 샘플 수인 min_samples_leaf를 결정하여 최종 파라미터 결정

In [33]:
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [36]:
for i in min_samples_split:
    min_samples_split_num = int(i)
    rf = RandomForestClassifier(max_depth=100,
                            n_estimators=1950,
                            max_features =75,
                            min_samples_split = min_samples_split_num,
                            n_jobs=-1, random_state = 201) #사용할 모델
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    f1 = f1_score(y_pred, y_test, average='weighted')
    
    print('min_samples_split : {0} & F1 score : {1}' .format(min_samples_split_num, f1))

min_samples_split : 2 & F1 score : 0.7362587985536095
min_samples_split : 5 & F1 score : 0.736840050798938
min_samples_split : 10 & F1 score : 0.7375401205558921


In [46]:
min_samples_split = [15, 20, 25, 30]

In [47]:
for i in min_samples_split:
    min_samples_split_num = int(i)
    rf = RandomForestClassifier(max_depth=100,
                            n_estimators=1950,
                            max_features =75,
                            min_samples_split = min_samples_split_num,
                            n_jobs=-1, random_state = 201) #사용할 모델
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    f1 = f1_score(y_pred, y_test, average='weighted')
    
    print('min_samples_split : {0} & F1 score : {1}' .format(min_samples_split_num, f1))

min_samples_split : 15 & F1 score : 0.7383601364669425
min_samples_split : 20 & F1 score : 0.7368171421337554
min_samples_split : 25 & F1 score : 0.7358046228858647


KeyboardInterrupt: 

In [48]:
min_samples_split = [16, 17, 18]

In [49]:
for i in min_samples_split:
    min_samples_split_num = int(i)
    rf = RandomForestClassifier(max_depth=100,
                            n_estimators=1950,
                            max_features =75,
                            min_samples_split = min_samples_split_num,
                            n_jobs=-1, random_state = 201) #사용할 모델
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    f1 = f1_score(y_pred, y_test, average='weighted')
    
    print('min_samples_split : {0} & F1 score : {1}' .format(min_samples_split_num, f1))

min_samples_split : 16 & F1 score : 0.7379079926111103
min_samples_split : 17 & F1 score : 0.7370232080916431
min_samples_split : 18 & F1 score : 0.7372181482694448


In [50]:
min_samples_leaf = [1, 2, 4]

In [53]:
for i in min_samples_leaf:
    min_samples_leaf_num = int(i)
    rf = RandomForestClassifier(max_depth=100,
                            n_estimators=1950,
                            max_features =75,
                            min_samples_split = 15,
                            min_samples_leaf = min_samples_leaf_num,
                            n_jobs=-1, random_state = 201) #사용할 모델
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    f1 = f1_score(y_pred, y_test, average='weighted')
    
    print('min_samples_leaf_num : {0} & F1 score : {1}' .format(min_samples_split_num, f1))

min_samples_leaf_num : 18 & F1 score : 0.7383601364669425
min_samples_leaf_num : 18 & F1 score : 0.7391481800750069
min_samples_leaf_num : 18 & F1 score : 0.7360127953228047


In [54]:
min_samples_leaf = [3, 5, 7]

In [55]:
for i in min_samples_leaf:
    min_samples_leaf_num = int(i)
    rf = RandomForestClassifier(max_depth=100,
                            n_estimators=1950,
                            max_features =75,
                            min_samples_split = 15,
                            min_samples_leaf = min_samples_leaf_num,
                            n_jobs=-1, random_state = 201) #사용할 모델
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    f1 = f1_score(y_pred, y_test, average='weighted')
    
    print('min_samples_leaf_num : {0} & F1 score : {1}' .format(min_samples_leaf_num, f1))

min_samples_leaf_num : 3 & F1 score : 0.7373818247414129
min_samples_leaf_num : 5 & F1 score : 0.7357161377549972
min_samples_leaf_num : 7 & F1 score : 0.7332354024528538


In [ ]:
for i in min_samples_leaf:
    min_samples_leaf_num = int(i)
    rf = RandomForestClassifier(max_depth=100,
                            n_estimators=1950,
                            max_features =75,
                            min_samples_split = 15,
                            min_samples_leaf = min_samples_leaf_num,
                            n_jobs=-1, random_state = 201) #사용할 모델
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    f1 = f1_score(y_pred, y_test, average='weighted')
    
    print('min_samples_leaf_num : {0} & F1 score : {1}' .format(min_samples_split_num, f1))

# Real Last and Best tune

In [58]:
rf = RandomForestClassifier(max_depth=100,
                            n_estimators=1950,
                            max_features =75,
                            min_samples_split = 15,
                            min_samples_leaf = 2,
                            bootstrap = False,
                            n_jobs=-1, random_state = 201) #사용할 모델
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
f1 = f1_score(y_pred, y_test, average='weighted')
f1

0.7430061331827581

In [60]:
accuracy_score(y_pred, y_test)

0.7392

In [62]:
print(classification_report(y_pred, y_test))

             precision    recall  f1-score   support

     2month       0.67      0.64      0.65      7830
      month       0.56      0.69      0.62      6131
   retained       0.83      0.77      0.80      8077
       week       0.89      0.84      0.87      7962

avg / total       0.75      0.74      0.74     30000



# Test Prediction

In [6]:
X_train = train.drop(['acc_id', 'label'], axis=1).values
y_train = train.label.values

In [7]:
X_train.shape

(100000, 585)

In [8]:
y_train.shape

(100000,)

In [10]:
X_test = test.drop('acc_id', axis=1).values

In [11]:
X_test.shape

(40000, 585)

In [12]:
rf = RandomForestClassifier(max_depth=100,
                            n_estimators=1950,
                            max_features =75,
                            min_samples_split = 15,
                            min_samples_leaf = 2,
                            bootstrap = False,
                            n_jobs=-1, random_state = 201) #사용할 모델
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

# Data Merge

In [13]:
result = pd.DataFrame()
result['acc_id'] = test.acc_id
result['label'] = y_pred

# Label distribution

In [16]:
result.groupby('label')['label'].count()

label
2month      10219
month        8237
retained    10777
week        10767
Name: label, dtype: int64

# Data Sorting

In [17]:
os.chdir("C:\\Users\\korea\\Downloads")
order = pd.read_csv('test_id.csv')
last = pd.merge(order, result, on='acc_id', how='inner')

# Data Saving

In [18]:
last.to_csv("C:\\Users\\korea\\Desktop\\Team_level0_predction1.csv")